In [1]:
#!pip install timm

In [2]:
import pandas as pd
import timm
import numpy as np
import json
import gc
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import albumentations
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
import torchvision
from typing import Any, Dict, List, Union, Optional
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import cv2
import torch
#from efficientnet_pytorch import EfficientNet
import numpy as np

from PIL import Image
from PIL import ImageFile

In [3]:
import pytorch_lightning
pytorch_lightning.seed_everything(42)

42

In [4]:
base_dir = r'C:\Users\Kaggle\Leaf_Classification\input'

In [5]:
### read csv and label mapping

labels = pd.read_csv(os.path.join(base_dir,'train.csv'))

with open(f'{base_dir}/label_num_to_disease_map.json') as f:
    label_mapper = json.load(f)

### Dataset class

In [6]:
class ImageClassificationDataset(Dataset):
    def __init__(
        self,
        image_names: List,
        transforms: Compose,
        labels: Optional[List[int]],
        img_path: str = '',
        mode: str = 'train',
        labels_to_ohe: bool = False,
        n_classes: int = 5,
    ):
        """
        Image classification dataset.

        Args:
            df: dataframe with image id and bboxes
            mode: train/val/test
            img_path: path to images
            transforms: albumentations
        """

        self.mode = mode
        self.transforms = transforms
        self.img_path = img_path
        self.image_names = image_names
        if labels is not None:
            if not labels_to_ohe:
                self.labels = np.array(labels)
            else:
                self.labels = np.zeros((len(labels), n_classes))
                self.labels[np.arange(len(labels)), np.array(labels)] = 1

    def __getitem__(self, idx: int) -> Dict[str, np.array]:
        image_path = self.img_path + self.image_names[idx]
        image = cv2.imread(f'{image_path}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is None:
            raise FileNotFoundError(image_path)
        target = self.labels[idx]

        img = self.transforms(image=image)['image']
        sample = {'image_path': image_path, 'image': img, 'target': np.array(target).astype('int64')}

        return sample

    def __len__(self) -> int:
        return len(self.image_names)

### Augs

In [7]:
sz = 576
sz_efffnetb4 = 600
deit_size = 384

In [8]:
TTA0_deit = albumentations.Compose([
            albumentations.Resize(deit_size, deit_size,p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA1_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.HorizontalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.VerticalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2_5_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.Transpose(p=1.),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA3_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=1
            ),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA4_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=1.
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA5_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.Cutout(p=1.), 
            ToTensorV2()],
            p=1.)

TTA6_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.ShiftScaleRotate(p=1.),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA7_deit = albumentations.Compose([
            albumentations.RandomResizedCrop(deit_size, deit_size),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=1.),
            ToTensorV2()],
            p=1.)

In [9]:
TTA0 = albumentations.Compose([
            albumentations.Resize(sz, sz,p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA1 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.HorizontalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.VerticalFlip(p=1),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA2_5 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Transpose(p=1.),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA3 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=1
            ),
           albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA4 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=1.
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)


TTA5 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.Cutout(p=1.), 
            ToTensorV2()],
            p=1.)

TTA6 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.ShiftScaleRotate(p=1.),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            ToTensorV2()],
            p=1.)

TTA7 = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=1.),
            ToTensorV2()],
            p=1.)

In [10]:
infer_augs_effnet = albumentations.Compose([
            albumentations.RandomResizedCrop(sz_efffnetb4, sz_efffnetb4),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)


infer_augs_resnext = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)

### PL module

In [11]:
def freeze_until(net: Any, param_name: str = None) -> None:
    """
    Freeze net until param_name


    Args:
        net:
        param_name:

    """
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name

class BasicEncoder(nn.Module):
    def __init__(
        self,
        arch: str = 'resnet18',
        source: str = 'torchvision',
        pretrained: str = None,
        n_layers: int = -1,
        freeze: bool = False,
        to_one_channel: bool = False,
        freeze_until_layer: str = None,
    ) -> None:
        """
        Initialize Encoder.

        Args:
            num_classes: the number of target classes, the size of the last layer's output
            arch: the name of the architecture form pretrainedmodels
            pretrained: the mode for pretrained model from pretrainedmodels
            n_layers: number of layers to keep
            freeze: to freeze model
            freeze_until: freeze until this layer. If None, then freeze all layers
        """
        super().__init__()
        if source == 'timm':
            net = timm.create_model(arch, pretrained=pretrained)
            self.output_dimension = list(net.children())[-1].in_features
        else:
            print("Not implemented")
            
        if freeze:
            freeze_until(net, freeze_until_layer)
        
        layers = list(net.children())[:n_layers]    
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        output = self.layers(x)

        return output

In [12]:
class BasicDecoder(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Linear(output_dimension , n_classes)

    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [13]:
class BasicDecoder_newhead(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Sequential(
            nn.Linear(output_dimension, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )

    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output


In [14]:
class Net_B4_newhead(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b4_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [15]:
class Net_B3(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b3_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [16]:
class Net_B4(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b4_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [17]:
class Net_B5(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b5_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [18]:
class BasicDecoder_seresnext(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        #self.fc = nn.Linear(output_dimension , n_classes)
        self.fc = nn.Sequential(
            nn.Linear(output_dimension, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )
    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [19]:
class seResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='seresnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_seresnext(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [20]:
class ResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [21]:
class ResnextNet50d(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnext50d_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [22]:
class resnest50d(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnest50d_1s4x24d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_newhead(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [23]:
class DEIT(nn.Module):
    def __init__(self) -> None:
        
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_384', pretrained=True)
        output_dimension = list(self.encoder.children())[-1].in_features

        self.encoder.head = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )
        self.loss = nn.CrossEntropyLoss()
    def forward(self, x, targets):
        logits = self.encoder(x)
        loss = self.loss(logits, targets).view(1)
        return logits, loss


In [24]:
class LitCassava(pl.LightningModule):
    def __init__(self, model):
        super(LitCassava, self).__init__()
        self.model = model
        self.metric = pl.metrics.Accuracy()
        self.learning_rate = 1e-4

    def forward(self, x, targets, *args, **kwargs):
        return self.model(x, targets)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

        return (
            [optimizer],
            [{'scheduler': scheduler, 'interval': 'epoch', 'monitor': 'valid_loss'}],
        )

    def training_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'train_loss': loss, f'train_accuracy': score}
        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'train_accuracy': score,
        }

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)
        
        logs = {'train_loss': avg_loss, 'train_accuracy': score}
        return {'log': logs, 'progress_bar': logs}

    def validation_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'valid_loss': loss, f'valid_accuracy': score}

        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'valid_accuracy': score,
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)

        # score = torch.tensor(1.0, device=self.device)
        logs = {'valid_loss': avg_loss, f'valid_accuracy': score, 'accuracy': score}
        return {'valid_loss': avg_loss, 'log': logs, 'progress_bar': logs}

In [25]:
labels.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [26]:
# path = "../input/cassava-leaf-disease-classification/"
# sub = pd.read_csv(f'{path}/sample_submission.csv')
# sub.head()

In [27]:
def getloader(df,path,bs=8,fold=0,tta=0,deit=False):
    
    if not deit:
        if tta == 0:
            augs = TTA0
        if tta == 1:
            augs = TTA1
        if tta == 2:
            augs = TTA2
        if tta == 3:
            augs = TTA2_5
        if tta == 4:
            augs = TTA3
        if tta == 5:
            augs = TTA4
        if tta == 6:
            augs = TTA5
        if tta == 7:
            augs = TTA6
        if tta == 8:
            augs = TTA7
    else:
        if tta == 0:
            augs = TTA0_deit
        if tta == 1:
            augs = TTA1_deit
        if tta == 2:
            augs = TTA2_deit
        if tta == 3:
            augs = TTA2_5_deit
        if tta == 4:
            augs = TTA3_deit
        if tta == 5:
            augs = TTA4_deit
        if tta == 6:
            augs = TTA5_deit
        if tta == 7:
            augs = TTA6_deit
        if tta == 8:
            augs = TTA7_deit
    
    print(f"Using TTA: {tta} and DEIT:{deit}")
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    train_indexes, valid_indexes = list(folds.split(df, df['label']))[fold]

    train_df = df.iloc[train_indexes]
    valid_df = df.iloc[valid_indexes]


    train_dataset = ImageClassificationDataset(image_names=train_df['image_id'].values,
                                                    transforms=augs,
                                                    labels=train_df['label'].values,
                                                    img_path=path,
                                                    mode='train',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    valid_dataset = ImageClassificationDataset(image_names=valid_df['image_id'].values,
                                                    transforms=augs,
                                                    labels=valid_df['label'].values,
                                                    img_path=path,
                                                    mode='valid',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    
    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=bs,num_workers=0,shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=bs,num_workers=0,shuffle=False)

    return train_loader,valid_loader,valid_indexes

In [28]:
# models = os.listdir('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss/')
# modelsPath = [os.path.join('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss',i) for i in models]
# modelsPath

In [29]:
from tqdm import tqdm
def getPreds(test_loader,lit_model,logitsOut=True):
    predictions = []
    
    for batch in tqdm(test_loader):
        image = batch['image'].to('cuda')
        target = batch['target'].to('cuda')
        with torch.no_grad():
            outputs = lit_model.model(image, target)[0]
            _outputs = outputs.softmax(1).detach().cpu().numpy()
            preds = outputs.argmax(1).detach().cpu().numpy()
            if not logitsOut:
                predictions.append(preds)
            else:
                #predictions[range(len(_outputs))] += _outputs
                predictions.append(_outputs)
    return np.array(predictions)

### Preds for Effnet b4 Distilled 2 SWA

In [39]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_effnetB4_distillation\avgWeights\_Avg_fold_upd_4.ckpt'
             }

In [40]:
checkpoint = torch.load(modelsPath[0], map_location=lambda storage, loc: storage)
checkpoint.keys()

odict_keys(['model.encoder.layers.0.weight', 'model.encoder.layers.1.weight', 'model.encoder.layers.1.bias', 'model.encoder.layers.1.running_mean', 'model.encoder.layers.1.running_var', 'model.encoder.layers.1.num_batches_tracked', 'model.encoder.layers.3.0.0.conv_dw.weight', 'model.encoder.layers.3.0.0.bn1.weight', 'model.encoder.layers.3.0.0.bn1.bias', 'model.encoder.layers.3.0.0.bn1.running_mean', 'model.encoder.layers.3.0.0.bn1.running_var', 'model.encoder.layers.3.0.0.bn1.num_batches_tracked', 'model.encoder.layers.3.0.0.se.conv_reduce.weight', 'model.encoder.layers.3.0.0.se.conv_reduce.bias', 'model.encoder.layers.3.0.0.se.conv_expand.weight', 'model.encoder.layers.3.0.0.se.conv_expand.bias', 'model.encoder.layers.3.0.0.conv_pw.weight', 'model.encoder.layers.3.0.0.bn2.weight', 'model.encoder.layers.3.0.0.bn2.bias', 'model.encoder.layers.3.0.0.bn2.running_mean', 'model.encoder.layers.3.0.0.bn2.running_var', 'model.encoder.layers.3.0.0.bn2.num_batches_tracked', 'model.encoder.layer

In [41]:
preds_effnet_b4_distilled = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#     lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4_distilled[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:26<00:00,  1.56it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:32<00:00,  1.44it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:32<00:00,  1.45it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:38<00:00,  1.36it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [53]:
preds_mean_effnet_b4_distilled = preds_effnet_b4_distilled.copy()
preds_mean_effnet_b4_distilled[:,0]= preds_effnet_b4_distilled[:,0]/(1+np.sqrt(0.05))
preds_mean_effnet_b4_distilled[:,1]= preds_effnet_b4_distilled[:,1]/(1+np.sqrt(0.102))
preds_mean_effnet_b4_distilled[:,2]= preds_effnet_b4_distilled[:,2]/(1+np.sqrt(0.111))
preds_mean_effnet_b4_distilled[:,3]= preds_effnet_b4_distilled[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnet_b4_distilled[:,4]= preds_effnet_b4_distilled[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnet_b4_distilled,1)

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4_distilled,1))

(0.8993784175351685, 0.8991447399168108)

In [55]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Preds for DEIT

In [31]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\DEIT_Mixup_without_hesitation_After_Epoch10\avgWeights\_Avg_fold_4.ckpt'}
             


In [32]:
preds_DEIT = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = DEIT()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta,deit=True)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_DEIT[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


Using cache found in C:\Users\Rajneesh Tiwari/.cache\torch\hub\facebookresearch_deit_main
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:True


100%|██████████| 134/134 [01:08<00:00,  1.97it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:True


...

100%|██████████| 134/134 [01:07<00:00,  1.98it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:True


100%|██████████| 134/134 [01:06<00:00,  2.01it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:True


100%|██████████| 134/134 [01:10<00:00,  1.91it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [33]:
preds_mean_DEIT = preds_DEIT.copy()
preds_mean_DEIT[:,0]= preds_DEIT[:,0]/(1+np.sqrt(0.05))
preds_mean_DEIT[:,1]= preds_DEIT[:,1]/(1+np.sqrt(0.102))
preds_mean_DEIT[:,2]= preds_DEIT[:,2]/(1+np.sqrt(0.111))
preds_mean_DEIT[:,3]= preds_DEIT[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_DEIT[:,4]= preds_DEIT[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_DEIT,1)

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_DEIT,1))

(0.8976024676356499, 0.8973220544936206)

In [35]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

311

### Preds for effnet B3

In [36]:
#modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\'')

modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b3_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'}
             


In [37]:
# modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\)

# modelsPath = [os.path.join(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\best_ones',i) for i in models]
# modelsPath

In [38]:
preds_effnet_b3 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B3()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b3[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:17<00:00,  1.74it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:16<00:00,  1.75it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:16<00:00,  1.76it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:22<00:00,  1.62it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [39]:
preds_mean_effnetb3 = preds_effnet_b3.copy()
preds_mean_effnetb3[:,0]= preds_effnet_b3[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb3[:,1]= preds_effnet_b3[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb3[:,2]= preds_effnet_b3[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb3[:,3]= preds_effnet_b3[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb3[:,4]= preds_effnet_b3[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb3,1)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b3,1))

(0.8969949058279199, 0.897181847922606)

In [41]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

340

### Preds for seresnext

In [42]:
#modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\'')

modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }


In [43]:
# modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\)

# modelsPath = [os.path.join(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\best_ones',i) for i in models]
# modelsPath

In [44]:
preds_seresnext = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = seResnextNet()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=128,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_seresnext[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 34/34 [01:40<00:00,  2.97s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 34/34 [01:47<00:00,  3.16s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 34/34 [01:46<00:00,  3.13s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/34 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 34/34 [01:52<00:00,  3.31s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [45]:
preds_mean_seresnext = preds_seresnext.copy()
preds_mean_seresnext[:,0]= preds_mean_seresnext[:,0]/(1+np.sqrt(0.05))
preds_mean_seresnext[:,1]= preds_mean_seresnext[:,1]/(1+np.sqrt(0.102))
preds_mean_seresnext[:,2]= preds_mean_seresnext[:,2]/(1+np.sqrt(0.111))
preds_mean_seresnext[:,3]= preds_mean_seresnext[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_seresnext[:,4]= preds_mean_seresnext[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_seresnext,1)

In [46]:
# classOut = np.argmax(preds_seresnext,1)
# classOut.shape

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_seresnext,1))

(0.8992382109641539, 0.8996120951535262)

In [48]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

360

In [49]:
torch.cuda.empty_cache()

In [50]:
gc.collect()

40

### Preds for Effnet b4 Taylor Loss - 2 SWA

In [51]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\Top 2\avgWeights\_Avg_fold_4.ckpt'
             }

In [52]:
preds_effnet_b4_taylor = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4_taylor[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:27<00:00,  1.54it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:36<00:00,  1.39it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:35<00:00,  1.40it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:42<00:00,  1.31it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [53]:
preds_mean_effnetb4_taylor = preds_effnet_b4_taylor.copy()
preds_mean_effnetb4_taylor[:,0]= preds_effnet_b4_taylor[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4_taylor[:,1]= preds_effnet_b4_taylor[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4_taylor[:,2]= preds_effnet_b4_taylor[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4_taylor[:,3]= preds_effnet_b4_taylor[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4_taylor[:,4]= preds_effnet_b4_taylor[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4_taylor,1)

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4_taylor,1))

(0.8993784175351685, 0.8991447399168108)

In [55]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Preds for Effnet b4 Taylor Loss - 3 SWA

In [127]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }


In [128]:
preds_effnet_b4_taylor_3SWA = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4_taylor_3SWA[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:25<00:00,  1.57it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:24<00:00,  1.58it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:24<00:00,  1.59it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:30<00:00,  1.48it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [129]:
preds_mean_effnetb4_taylor_3SWA = preds_effnet_b4_taylor_3SWA.copy()
preds_mean_effnetb4_taylor_3SWA[:,0]= preds_effnet_b4_taylor_3SWA[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4_taylor_3SWA[:,1]= preds_effnet_b4_taylor_3SWA[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4_taylor_3SWA[:,2]= preds_effnet_b4_taylor_3SWA[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4_taylor_3SWA[:,3]= preds_effnet_b4_taylor_3SWA[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4_taylor_3SWA[:,4]= preds_effnet_b4_taylor_3SWA[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4_taylor_3SWA,1)

In [130]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4_taylor_3SWA,1))

(0.8976024676356499, 0.8978828807776791)

In [131]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Preds for B4 focal cosine loss

In [56]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }


In [57]:
preds_effnet_b4 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):    
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4_newhead()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)      
#    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:25<00:00,  1.56it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:24<00:00,  1.58it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:24<00:00,  1.59it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:30<00:00,  1.47it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [58]:
preds_mean_effnetb4 = preds_effnet_b4.copy()
preds_mean_effnetb4[:,0]= preds_effnet_b4[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4[:,1]= preds_effnet_b4[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4[:,2]= preds_effnet_b4[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4[:,3]= preds_effnet_b4[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4[:,4]= preds_effnet_b4[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4,1)

In [59]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4,1))

(0.8985371781090807, 0.8989110622984531)

In [60]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Ensemble b4 with deit

In [61]:
res_ = pd.DataFrame()
res_['b4_contrib'] = 0
#res['b5_contrib'] = 0
res_['deit_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    deit_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4+deit_contrib*preds_DEIT)
    res_.loc[count,'b4_contrib'] = i
    res_.loc[count,'deit_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [62]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,deit_contrib,Accuracy
61,0.61,0.39,0.903164
54,0.54,0.46,0.903071
62,0.62,0.38,0.903071
59,0.59,0.41,0.902930
63,0.63,0.37,0.902930
...,...,...,...
4,0.04,0.96,0.897883
3,0.03,0.97,0.897696
1,0.01,0.99,0.897509
2,0.02,0.98,0.897509


### Ensemble 

In [63]:
res_ = pd.DataFrame()
res_['b4_contrib'] = 0
#res['b5_contrib'] = 0
res_['seresnext_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    seresnext_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext)
    res_.loc[count,'b4_contrib'] = i
    res_.loc[count,'seresnext_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [64]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,seresnext_contrib,Accuracy
11,0.11,0.89,0.902369
17,0.17,0.83,0.902276
16,0.16,0.84,0.902276
12,0.12,0.88,0.902183
21,0.21,0.79,0.902183
...,...,...,...
99,0.99,0.01,0.898911
95,0.95,0.05,0.898864
96,0.96,0.04,0.898771
97,0.97,0.03,0.898771


### Ensemble deit with b4 taylor loss

In [65]:
res_ = pd.DataFrame()
res_['b4_taylor_contrib'] = 0
#res['b5_contrib'] = 0
res_['deit_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    deit_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4_taylor+deit_contrib*preds_DEIT)
    res_.loc[count,'b4_taylor_contrib'] = i
    res_.loc[count,'deit_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [66]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_taylor_contrib,deit_contrib,Accuracy
37,0.37,0.63,0.902556
38,0.38,0.62,0.902510
32,0.32,0.68,0.902463
31,0.31,0.69,0.902463
39,0.39,0.61,0.902416
...,...,...,...
4,0.04,0.96,0.898210
3,0.03,0.97,0.897930
2,0.02,0.98,0.897743
1,0.01,0.99,0.897416


### Ensemble seresnext with b4 taylor loss

In [67]:
res_ = pd.DataFrame()
res_['b4_taylor_contrib'] = 0
#res['b5_contrib'] = 0
res_['seresnext_contrib'] = 0
res_ ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    b4_contrib = i
    seresnext_contrib = 1-i
    #print(i,j,1-i-j)
    ens_preds = (b4_contrib*preds_effnet_b4_taylor+seresnext_contrib*preds_seresnext)
    res_.loc[count,'b4_taylor_contrib'] = i
    res_.loc[count,'seresnext_contrib'] = 1-i
    res_.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
    count += 1

In [68]:
res_.sort_values(by=['Accuracy'],ascending=False)

,b4_taylor_contrib,seresnext_contrib,Accuracy
13,0.13,0.87,0.902743
12,0.12,0.88,0.902743
11,0.11,0.89,0.902556
14,0.14,0.86,0.902510
15,0.15,0.85,0.902463
...,...,...,...
94,0.94,0.06,0.899285
96,0.96,0.04,0.899285
98,0.98,0.02,0.899285
95,0.95,0.05,0.899238


### Ensemble deit+seresnext+b4 focal cosine

In [69]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['deit_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0
res ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        deit_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+deit_contrib*preds_mean_DEIT)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+deit_contrib*preds_DEIT)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'deit_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
        count += 1

In [70]:
res.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,deit_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
2519,0.29,0.25,0.46,0.904145,0.903678
2447,0.28,0.25,0.47,0.904145,0.903772
2520,0.29,0.26,0.45,0.904099,0.903678
2663,0.31,0.28,0.41,0.904099,0.903725
2373,0.27,0.24,0.49,0.904099,0.903444
...,...,...,...,...,...
198,0.01,0.98,0.01,0.897509,0.897836
97,0.00,0.97,0.03,0.897509,0.897696
296,0.02,0.97,0.01,0.897509,0.897602
98,0.00,0.98,0.02,0.897462,0.897602


In [71]:
res.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,deit_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
2449,0.28,0.27,0.45,0.903818,0.904005
2522,0.29,0.28,0.43,0.903865,0.903958
3248,0.40,0.28,0.32,0.903631,0.903958
2664,0.31,0.29,0.40,0.903772,0.903958
2594,0.30,0.29,0.41,0.903725,0.903912
...,...,...,...,...,...
97,0.00,0.97,0.03,0.897509,0.897696
96,0.00,0.96,0.04,0.897556,0.897649
99,0.00,0.99,0.01,0.897322,0.897602
98,0.00,0.98,0.02,0.897462,0.897602


### Preds for B5

In [72]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\avgWeights\_Avg_fold_4.ckpt'
             }
             


In [73]:
preds_effnet_b5 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B5()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b5[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:35<00:00,  1.41it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:36<00:00,  1.39it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:35<00:00,  1.40it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:42<00:00,  1.31it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [74]:
preds_mean_effnetb5 = preds_effnet_b5.copy()
preds_mean_effnetb5[:,0]= preds_effnet_b5[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb5[:,1]= preds_effnet_b5[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb5[:,2]= preds_effnet_b5[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb5[:,3]= preds_effnet_b5[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb5[:,4]= preds_effnet_b5[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb5,1)

In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b5,1))

(0.8969014347805767, 0.8987241202037669)

In [76]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Ensemble

In [77]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['b5_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0
res ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        b5_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'b5_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
        count += 1

In [78]:
pd.set_option('display.max_rows',1000)

In [79]:
res.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
1668,0.18,0.21,0.61,0.904613,0.903678
1504,0.16,0.24,0.60,0.904473,0.903538
1669,0.18,0.22,0.60,0.904473,0.903818
1492,0.16,0.12,0.72,0.904473,0.903912
1409,0.15,0.14,0.71,0.904426,0.903865
...,...,...,...,...,...
5043,0.95,0.00,0.05,0.898864,0.898864
5053,0.97,0.00,0.03,0.898771,0.898864
5048,0.96,0.00,0.04,0.898771,0.898818
99,0.00,0.99,0.01,0.898771,0.896901


In [80]:
res.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
1152,0.12,0.18,0.70,0.903865,0.904192
1240,0.13,0.18,0.69,0.903772,0.904192
1065,0.11,0.20,0.69,0.903538,0.904192
1153,0.12,0.19,0.69,0.903678,0.904145
1326,0.14,0.17,0.69,0.904052,0.904099
...,...,...,...,...,...
197,0.01,0.97,0.02,0.899098,0.897042
198,0.01,0.98,0.01,0.899051,0.896948
98,0.00,0.98,0.02,0.898911,0.896948
97,0.00,0.97,0.03,0.898958,0.896948


### resnest50d_1s4x24d

In [81]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnest50d_1s4x24d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }
             


In [82]:
preds_resnest50d = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = resnest50d()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_resnest50d[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:25<00:00,  1.56it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:27<00:00,  1.54it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:26<00:00,  1.55it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:33<00:00,  1.44it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [83]:
preds_mean_resnest50d = preds_resnest50d.copy()
preds_mean_resnest50d[:,0]= preds_resnest50d[:,0]/(1+np.sqrt(0.05))
preds_mean_resnest50d[:,1]= preds_resnest50d[:,1]/(1+np.sqrt(0.102))
preds_mean_resnest50d[:,2]= preds_resnest50d[:,2]/(1+np.sqrt(0.111))
preds_mean_resnest50d[:,3]= preds_resnest50d[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_resnest50d[:,4]= preds_resnest50d[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_resnest50d,1)

In [84]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_resnest50d,1))

(0.8951254848810581, 0.8979763518250222)

### Resnext50D

In [85]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_0.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_1.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_2.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_3.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\resnext50_32x4d_sz_576_Mixup_0.3_timm_newhead\avgWeights\_Avg_fold_4.ckpt'
             }
             


In [86]:
preds_resnext50d = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = ResnextNet50d()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
#     lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.load_state_dict(checkpoint)  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        _,test_loader,valid_ix = getloader(labels,path=train_img_path,bs=32,fold=f,tta=tta)
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_resnext50d[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 0 and DEIT:False


100%|██████████| 134/134 [01:35<00:00,  1.40it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 1 and DEIT:False


...

100%|██████████| 134/134 [01:36<00:00,  1.38it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 3 and DEIT:False


100%|██████████| 134/134 [01:36<00:00,  1.39it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  0%|          | 0/134 [00:00<?, ?it/s]

Using TTA: 4 and DEIT:False


100%|██████████| 134/134 [01:43<00:00,  1.30it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [87]:
preds_mean_resnext50d = preds_resnext50d.copy()
preds_mean_resnext50d[:,0]= preds_resnext50d[:,0]/(1+np.sqrt(0.05))
preds_mean_resnext50d[:,1]= preds_resnext50d[:,1]/(1+np.sqrt(0.102))
preds_mean_resnext50d[:,2]= preds_resnext50d[:,2]/(1+np.sqrt(0.111))
preds_mean_resnext50d[:,3]= preds_resnext50d[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_resnext50d[:,4]= preds_resnext50d[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_resnext50d,1)

In [88]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_resnest50d,1))

(0.8946581296443427, 0.8979763518250222)

### Ensemble

In [89]:
res_all4 = pd.DataFrame()
res_all4['b4_contrib'] = 0
res_all4['b5_contrib'] = 0
res_all4['seresnext_contrib'] = 0
res_all4['resnest_contrib'] = 0

res_all4 ['Accuracy'] = 0
res_all4 ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.05):
    for j in np.arange(0,1-i,0.05):
        for k in np.arange(0,1-i-j,0.05):
            b4_contrib = i
            b5_contrib = j
            seresnext_contrib = k
            resnest_contrib = 1-i-j-k
            print(b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib)
            #assert b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib==1.
            #print(i,j,k,1-(i+j+k))
            ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5+resnest_contrib*preds_mean_resnest50d)
            ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5+resnest_contrib*preds_resnest50d)
            res_all4.loc[count,'b4_contrib'] = i
            res_all4.loc[count,'seresnext_contrib'] = j
            res_all4.loc[count,'b5_contrib'] = k
            res_all4.loc[count,'resnest_contrib'] = 1-i-j-k
            res_all4.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
            res_all4.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
            count += 1

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0

In [90]:
res_all4.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,resnest_contrib,Accuracy,Adj_Accuracy
475,0.10,0.45,0.20,0.25,0.904473,0.903538
251,0.05,0.20,0.10,0.65,0.904473,0.904052
633,0.15,0.65,0.15,0.05,0.904473,0.904052
457,0.10,0.30,0.15,0.45,0.904473,0.904145
477,0.10,0.55,0.20,0.15,0.904379,0.904052
...,...,...,...,...,...,...
1548,0.95,0.00,0.00,0.05,0.899051,0.898771
207,0.00,0.00,0.90,0.10,0.899051,0.897275
1,0.00,0.05,0.00,0.95,0.898958,0.897042
209,0.00,0.00,0.95,0.05,0.898818,0.897182


In [91]:
res_all4.sort_values(by=['Adj_Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,resnest_contrib,Accuracy,Adj_Accuracy
239,0.05,0.50,0.05,0.40,0.903538,0.904660
463,0.10,0.60,0.15,0.15,0.903865,0.904613
238,0.05,0.45,0.05,0.45,0.903585,0.904566
257,0.05,0.50,0.10,0.35,0.903304,0.904519
446,0.10,0.55,0.10,0.25,0.904239,0.904473
...,...,...,...,...,...,...
204,0.00,0.00,0.85,0.15,0.899378,0.897275
207,0.00,0.00,0.90,0.10,0.899051,0.897275
209,0.00,0.00,0.95,0.05,0.898818,0.897182
1,0.00,0.05,0.00,0.95,0.898958,0.897042


In [93]:
#preds_DEIT
#preds_mean_DEIT

In [102]:
res_all4 = pd.DataFrame()
res_all4['DEIT_contrib'] = 0
res_all4['b5_contrib'] = 0
res_all4['seresnext_contrib'] = 0
res_all4['resnest_contrib'] = 0

res_all4 ['Accuracy'] = 0
res_all4 ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.05):
    for j in np.arange(0,1-i,0.05):
        for k in np.arange(0,1-i-j,0.05):
            DEIT_contrib = i
            b5_contrib = j 
            seresnext_contrib = k = 0 
            resnest_contrib = 1-i-j-k
            print(DEIT_contrib+b5_contrib+seresnext_contrib+resnest_contrib)
            #assert b4_contrib+b5_contrib+seresnext_contrib+resnest_contrib==1.
            #print(i,j,k,1-(i+j+k))
            ens_preds_adj = (DEIT_contrib*preds_mean_DEIT+seresnext_contrib*preds_mean_seresnext+b5_contrib*preds_mean_effnetb5+resnest_contrib*preds_mean_resnest50d)
            ens_preds = (DEIT_contrib*preds_DEIT+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5+resnest_contrib*preds_resnest50d)
            res_all4.loc[count,'DEIT_contrib'] = i
            res_all4.loc[count,'seresnext_contrib'] = j
            res_all4.loc[count,'b5_contrib'] = k
            res_all4.loc[count,'resnest_contrib'] = 1-i-j-k
            res_all4.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
            res_all4.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
            count += 1

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.999999

In [135]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_effnet_b4,1))

array([[1.        , 0.65706614],
       [0.65706614, 1.        ]])

In [136]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_effnet_b5,1))

array([[1.        , 0.71135543],
       [0.71135543, 1.        ]])

In [139]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_seresnext,1))

array([[1.        , 0.63602983],
       [0.63602983, 1.        ]])

In [140]:
np.corrcoef(np.max(preds_DEIT,1),np.max(preds_resnest50d,1))

array([[1.      , 0.554017],
       [0.554017, 1.      ]])

In [143]:
np.corrcoef(np.max(preds_seresnext,1),np.max(preds_resnest50d,1))

array([[1.        , 0.90363851],
       [0.90363851, 1.        ]])

In [144]:
np.corrcoef(np.max(preds_effnet_b4_taylor,1),np.max(preds_effnet_b4,1))

array([[1.        , 0.68386724],
       [0.68386724, 1.        ]])

In [105]:
res_all4.sort_values(by=['Accuracy'],ascending=False)

,DEIT_contrib,b5_contrib,seresnext_contrib,resnest_contrib,Accuracy,Adj_Accuracy
647,0.15,0.0,0.25,0.60,0.904379,0.903257
648,0.15,0.0,0.25,0.60,0.904379,0.903257
656,0.15,0.0,0.25,0.60,0.904379,0.903257
655,0.15,0.0,0.25,0.60,0.904379,0.903257
654,0.15,0.0,0.25,0.60,0.904379,0.903257
...,...,...,...,...,...,...
1540,0.85,0.0,0.00,0.15,0.897696,0.897789
1541,0.85,0.0,0.00,0.15,0.897696,0.897789
1545,0.90,0.0,0.00,0.10,0.897416,0.897602
1546,0.90,0.0,0.00,0.10,0.897416,0.897602


In [111]:
from sklearn.metrics import confusion_matrix

confusion_matrix(labels.label,np.argmax(preds_effnet_b4,1),normalize='true')

array([[0.64673413, 0.07175713, 0.02667893, 0.04967801, 0.20515179],
       [0.04339881, 0.81955231, 0.02329831, 0.04111466, 0.07263591],
       [0.00963956, 0.01969824, 0.81642917, 0.10393965, 0.05029338],
       [0.00151999, 0.00372397, 0.01170391, 0.97674419, 0.00630795],
       [0.06946061, 0.04928211, 0.04889406, 0.08071401, 0.7516492 ]])

In [120]:
confusion_matrix(labels.label,np.argmax(preds_mean_effnetb4,1),normalize='true')

array([[0.65409384, 0.07083717, 0.02575897, 0.04783809, 0.20147194],
       [0.04522613, 0.82275011, 0.02284148, 0.03883052, 0.07035176],
       [0.01131601, 0.01927913, 0.82020117, 0.0984912 , 0.05071249],
       [0.00182399, 0.00402797, 0.0130719 , 0.9743882 , 0.00668795],
       [0.0717889 , 0.04928211, 0.04967016, 0.07799767, 0.75126116]])

In [112]:
confusion_matrix(labels.label,np.argmax(preds_effnet_b4_taylor,1),normalize='true')

array([[0.66421343, 0.05979761, 0.02667893, 0.04047838, 0.20883165],
       [0.04751028, 0.80721791, 0.02649612, 0.04385564, 0.07492005],
       [0.01131601, 0.01886002, 0.82564962, 0.09639564, 0.04777871],
       [0.00167199, 0.00357197, 0.01322389, 0.9753002 , 0.00623195],
       [0.06790842, 0.04656577, 0.05277454, 0.07722158, 0.75552969]])

In [119]:
confusion_matrix(labels.label,np.argmax(preds_mean_effnetb4_taylor,1),normalize='true')

array([[0.67709292, 0.06071757, 0.02483901, 0.03863845, 0.19871205],
       [0.04842394, 0.80995889, 0.02695295, 0.04065783, 0.0740064 ],
       [0.01215423, 0.01886002, 0.82984074, 0.09178541, 0.0473596 ],
       [0.00220398, 0.00364797, 0.01443988, 0.97279222, 0.00691594],
       [0.07062476, 0.04695382, 0.05316259, 0.07062476, 0.75863407]])

In [113]:
confusion_matrix(labels.label,np.argmax(preds_DEIT,1),normalize='true')

array([[0.65961362, 0.07727691, 0.01563937, 0.04415823, 0.20331187],
       [0.04522613, 0.81361352, 0.02284148, 0.04385564, 0.07446323],
       [0.01257334, 0.02095557, 0.79128248, 0.11232188, 0.06286672],
       [0.00189998, 0.00395197, 0.00965192, 0.97803618, 0.00645995],
       [0.07411719, 0.04889406, 0.0411331 , 0.08110206, 0.75475359]])

In [114]:
confusion_matrix(labels.label,np.argmax(preds_effnet_b5,1),normalize='true')

array([[0.67433303, 0.06071757, 0.02115915, 0.04139834, 0.2023919 ],
       [0.04294198, 0.81589767, 0.02740978, 0.03654637, 0.0772042 ],
       [0.01341157, 0.01886002, 0.81810562, 0.09681475, 0.05280805],
       [0.00197598, 0.00417997, 0.01337589, 0.97378021, 0.00668795],
       [0.06752037, 0.04889406, 0.05316259, 0.07528133, 0.75514164]])

In [115]:
confusion_matrix(labels.label,np.argmax(preds_resnest50d,1),normalize='true')

array([[0.64305428, 0.07451702, 0.02115915, 0.04967801, 0.21159154],
       [0.04111466, 0.81955231, 0.0246688 , 0.04568296, 0.06898127],
       [0.01215423, 0.01969824, 0.79756915, 0.11860855, 0.05196982],
       [0.00144399, 0.00326797, 0.01025992, 0.97993616, 0.00509196],
       [0.06635623, 0.05044626, 0.05044626, 0.08614668, 0.74660458]])

In [117]:
confusion_matrix(labels.label,np.argmax(preds_resnext50d,1),normalize='true')

array([[0.63661454, 0.0625575 , 0.01839926, 0.0524379 , 0.2299908 ],
       [0.04522613, 0.81041571, 0.02421197, 0.04385564, 0.07629054],
       [0.01215423, 0.01760268, 0.80595138, 0.11232188, 0.05196982],
       [0.00205198, 0.00349597, 0.01162791, 0.97704818, 0.00577595],
       [0.06946061, 0.04811797, 0.04384944, 0.08614668, 0.7524253 ]])

In [118]:
confusion_matrix(labels.label,np.argmax(preds_seresnext,1),normalize='true')

array([[0.65777369, 0.0699172 , 0.01931923, 0.05335787, 0.19963201],
       [0.04385564, 0.81361352, 0.02649612, 0.04339881, 0.07263591],
       [0.01257334, 0.02011735, 0.79840738, 0.11944677, 0.04945516],
       [0.00167199, 0.00319197, 0.00835993, 0.98229214, 0.00448396],
       [0.07411719, 0.04617773, 0.04617773, 0.08731083, 0.74621653]])

In [125]:
b4_contrib = 0.18
seresnext_contrib = 0.61
b5_contrib = 0.21
_ = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5)
print(accuracy_score(labels.label,np.argmax(_,1)))
confusion_matrix(labels.label,np.argmax(_,1),normalize='true')

0.9046127961863812


array([[0.68445262, 0.06071757, 0.01839926, 0.04875805, 0.18767249],
       [0.04248515, 0.82366377, 0.02329831, 0.04202832, 0.06852444],
       [0.01257334, 0.01802179, 0.81098072, 0.11106454, 0.0473596 ],
       [0.00151999, 0.00334397, 0.00904393, 0.98130415, 0.00478796],
       [0.06480404, 0.04656577, 0.04772992, 0.07954986, 0.76135041]])

In [142]:
deit_contrib = 0.15
seresnext_contrib = 0.25
resnest_contrib = 0.6
_ = (deit_contrib*preds_DEIT+seresnext_contrib*preds_seresnext+resnest_contrib*preds_resnest50d)
print(accuracy_score(labels.label,np.argmax(_,1)))
confusion_matrix(labels.label,np.argmax(_,1),normalize='true')

0.9019488713371033


array([[0.67433303, 0.07083717, 0.01471941, 0.04967801, 0.19043238],
       [0.04431247, 0.82320694, 0.02147099, 0.04294198, 0.06806761],
       [0.01383068, 0.01969824, 0.79631182, 0.11567477, 0.05448449],
       [0.00159599, 0.00349597, 0.00835993, 0.98168415, 0.00486396],
       [0.07217695, 0.04617773, 0.04462553, 0.0814901 , 0.75552969]])

In [145]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['deit_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0
res ['Adj_Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        deit_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds_adj = (b4_contrib*preds_mean_effnetb4+seresnext_contrib*preds_mean_seresnext+deit_contrib*preds_mean_DEIT)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+deit_contrib*preds_DEIT)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'deit_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        res.loc[count,'Adj_Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds_adj,1))
        count += 1

In [147]:
res.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,deit_contrib,seresnext_contrib,Accuracy,Adj_Accuracy
2519,0.29,0.25,0.46,0.904145,0.903678
2447,0.28,0.25,0.47,0.904145,0.903772
2520,0.29,0.26,0.45,0.904099,0.903678
2663,0.31,0.28,0.41,0.904099,0.903725
2373,0.27,0.24,0.49,0.904099,0.903444
...,...,...,...,...,...
198,0.01,0.98,0.01,0.897509,0.897836
97,0.00,0.97,0.03,0.897509,0.897696
296,0.02,0.97,0.01,0.897509,0.897602
98,0.00,0.98,0.02,0.897462,0.897602


In [124]:
labels.label.value_counts()

3    13158
4     2577
2     2386
1     2189
0     1087
Name: label, dtype: int64

### Below looks good

In [148]:
deit_contrib = 0.25
seresnext_contrib = 0.46
b4_contrib = 0.29
_ = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+deit_contrib*preds_DEIT)
print(accuracy_score(labels.label,np.argmax(_,1)))
confusion_matrix(labels.label,np.argmax(_,1),normalize='true')

0.9041454409496659


array([[0.6826127 , 0.06439742, 0.01563937, 0.04691812, 0.19043238],
       [0.0447693 , 0.82046597, 0.02192782, 0.04294198, 0.06989493],
       [0.0108969 , 0.01927913, 0.80846605, 0.10896899, 0.05238894],
       [0.00151999, 0.00357197, 0.00919593, 0.98092415, 0.00478796],
       [0.06519208, 0.04617773, 0.04423749, 0.07916182, 0.76523089]])